In [6]:
# building a sample vectordb

from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

In [4]:
loader = TextLoader("speech.txt")
documents = loader.load()

In [8]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(documents)

In [9]:
embeddings = OllamaEmbeddings(model="llama3.2")
vectordb = Chroma.from_documents(splits, embeddings)
vectordb

/var/folders/yz/s57wsyt918j04kt6btpgwgy40000gn/T/ipykernel_55430/3938134608.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3.2")


In [10]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs = vectordb.similarity_search(query)
docs[0].page_content

'government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'

In [11]:
## saving to the disk
Chroma.from_documents(splits, embeddings, persist_directory="./chroma_db")

In [13]:
# load from disk
db2 = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

In [14]:
docs = db2.similarity_search(query)
docs[0].page_content

'government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'

In [15]:
# similarity search with score
docs = db2.similarity_search_with_score(query)
docs

[(Document(metadata={'source': 'speech.txt'}, page_content='government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'),
  6202.397460638273),
 (Document(metadata={'source': 'speech.txt'}, page_content='reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  6339.757849028646),
 (Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political libe

In [16]:
# retriever option
retreiver = db2.as_retriever()
retreiver.invoke(query)[0].page_content

'government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'